## Foundry Agents Grounded in Foundry IQ Knowledge Base

![lab_flow](./Assets/foundry_lab_flow.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
knowledge_base_name = os.getenv("KNOWLEDGE_BASE_NAME")
search_service_project_connection_name = os.getenv("SEARCH_SERVICE_PROJECT_CONNECTION_NAME")

# constructing the mcp endpoint
mcp_endpoint = f"{search_service_endpoint}/knowledgebases/{knowledge_base_name}/mcp?api-version=2025-11-01-preview"

### Setting Up the Foundry Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Getting the Search Connection ID

In [ ]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == search_service_project_connection_name:
        connection_id = connection.id
        break

print(f"The Search Service Connection ID is: {connection_id}")

### Creating the MCP Knowledge Base Tool

In [ ]:
mcp_kb_tool = MCPTool(
    server_label = knowledge_base_name,
    server_url = mcp_endpoint,
    require_approval = "never",
    allowed_tools = ["knowledge_base_retrieve"],
    project_connection_id = search_service_project_connection_name
)

### Defining the Agent System Prompt for Response Optimization

In [ ]:
instructions = """
You are a helpful assistant that must use the knowledge base to answer all the questions from user. You must never answer from your own knowledge under any circumstances.
At the end of your answer provide a list of all the document urls used as grounding sources.

If you cannot find the answer in the provided knowledge base you must respond with - I don't know.
"""

### Creating the Agent

In [ ]:
agent_name = "kb-code-agent"

agent = project_client.agents.create_version(
    agent_name = agent_name,
    definition = PromptAgentDefinition(
        model = model_deployment_name,
        instructions = instructions,
        tools = [mcp_kb_tool]
    )
)

print(f"Knowledge Base Agent Created with ID: {agent.id}")

### Create a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat with the Agent

Queries to try:
1) Combine the CEIM Impact Flux Score with the Well-Being Pillars Framework to propose a proprietary CarbonOps metric that quantifies how employee well-being influences ESG impact severity. Define the formula, list inputs required from SES metadata, and generate a sample score calculation.

2) Using the CarbonOps Global Remote Work tiers and the CEIM System Boundary Confidence concept, evaluate how a fully distributed team (RW-3) affects CarbonOps regulatory readiness. Provide a quantified readiness score and justify it using SES validation stages.

3) List all proprietary CarbonOps scoring elements (from CEIM, SES, and Well-Being documents) and classify them into: (1) metrics, (2) validation factors, (3) behavioral indicators. Explain how these categories interact during an AI-driven ESG assessment.

In [ ]:
user_query = f"""List all proprietary CarbonOps scoring elements (from CEIM, SES, and Well-Being documents) 
and classify them into: (1) metrics, (2) validation factors, (3) behavioral indicators. 
Explain how these categories interact during an AI-driven ESG assessment."""

In [ ]:
reponse = openai_client.responses.create(
    conversation=conversation.id,
    extra_body = {
        "agent": {
            "name": agent_name,
            "type": "agent_reference"
        }
    },
    input = user_query
)

print(f"Agent Response: {reponse.output_text}")